# Thesis random sweeps - 2Try - Fisher Info characterisation - Fixed noise

2/2

Same as 1try, but now fixed sigma_x=0.25 and sigma_baseline=0

Should give more useful samples for plots

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import scipy as sp
import scipy.stats as spst
import scipy.interpolate as spint
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.utils import io
import progress

from experimentlauncher import ExperimentLauncher
from dataio import DataIO
import plots_experimental_data
import em_circularmixture_parametrickappa


# import matplotlib.animation as plt_anim
from mpl_toolkits.mplot3d import Axes3D

import re
import inspect
import imp

import utils
import load_experimental_data

from plots_fitexperiment_papertheo import PlotsFitExperimentAllTPaperTheo

In [ ]:
with io.capture_output(display=False, stdout=True) as captured:
    %run reloader_fisher2016_random_large_2try_310816.py

In [ ]:
def avg_lastaxis(array_name, array):
    return [(array_name, utils.nanmean(array, axis=-1))]
def avg_twice_lastaxis(array_name, array):
    return [(array_name, utils.nanmean(utils.nanmean(array, axis=-1), axis=-1))]

def process_fi(array_name, array):
    outputs = avg_twice_lastaxis(array_name, array)
    outputs.extend(avg_twice_lastaxis(array_name + "_stddev", (2./array)**0.5))
    return outputs

def process_marginal_fi(array_name, array):
    # Marginal FI/Inv FI have (mean, std), just keep mean
    outputs_all = avg_lastaxis(array_name, array)
    
    if array_name.find('inv') > -1:
        outputs_all.extend(avg_lastaxis(array_name + "_stddev", (2.*array)**0.5))
    else:
        outputs_all.extend(avg_lastaxis(array_name + "_stddev", (2./array)**0.5))
    
    outputs = [(o[0], o[1][:, 0]) for o in outputs_all]
    return outputs

def process_em_fits(array_name, array):
    emfits_all = utils.nanmean(array, axis=-1)
    outputs = [(array_name + "_" + colname, emfits_all[:, col_i])
            for col_i, colname in enumerate(['kappa',
                                             'target',
                                             'nontargets',
                                             'random',
                                             'LL',
                                             'bic'])
              ]
    outputs.append((array_name + '_fidelity', 1./utils.kappa_to_stddev(emfits_all[:, 0])**2.))
    outputs.append((array_name + '_stddev', utils.kappa_to_stddev(emfits_all[:, 0])))
    
    return outputs

    
def construct_pandas_dataframe(data_pbs, pandas_columns_with_processing, num_repetitions):
    parameter_names_sorted = data_pbs.dataset_infos['parameters']
    filter_data = None
    result_parameters_flat = None

    pandas_column_data = []

    for result_array_name, result_processing in pandas_columns_with_processing:
        # Extract data
        res_array = np.array(data_pbs.dict_arrays[result_array_name]['results_flat'])

        # Filter completed only
        if filter_data is None:
            repeats_completed = data_pbs.dict_arrays[result_array_name]['repeats_completed']
            filter_data = repeats_completed == (num_repetitions - 1)
        res_array = res_array[filter_data]

        # Keep parameters
        if result_parameters_flat is None:
            result_parameters_flat = np.array(data_pbs.dict_arrays[result_array_name]['parameters_flat'])
            result_parameters_flat = result_parameters_flat[filter_data]

        # Transform into list of columns for Pandas
        pandas_column_data.extend(result_processing['process'](result_processing['name'], res_array))

    # Add all parameters to Pandas columns
    for param_i, param_name in enumerate(parameter_names_sorted):
        pandas_column_data.append((param_name, result_parameters_flat[:, param_i]))
    
    df_out = pd.DataFrame.from_items(pandas_column_data)
    
    # Remove NaN
    df_out = df_out.dropna()
    
    return df_out

In [ ]:
def remove_outliers(df, n_stddev=5):
    outliers = np.sum(np.abs(spst.zscore(df)) < n_stddev, axis=-1)
    return df[outliers >= outliers.max()]

In [ ]:
def df_add_quantize_parameters(df, parameters, nQuantiles):
    param_qbins = dict()
    param_qbins_middle = dict()

    for param_name in parameters:
        param_factored, param_qbins[param_name] = pd.qcut(df[param_name], nQuantiles, retbins=True, labels=False)
        param_qbins_middle[param_name] = ((param_qbins[param_name][:-1] + param_qbins[param_name][1:])/2.
                                         ).astype(df[param_name].dtype)
        df.loc[:, (param_name + "_qi")] = param_factored
    
    return df, param_qbins, param_qbins_middle

In [ ]:
def filter_dataframe(df, parameters_values):
    filter_mask = None
    for key, value in parameters_values.iteritems():
        new_filter = (df[key] == value)
        if filter_mask is None:
            filter_mask = new_filter
        else:
            filter_mask = filter_mask & new_filter
    
    if filter_mask is None:
        return df
    else:
        return df[filter_mask]

def filter_quantized_param(df, target_parameters, param_qbins):
    quantized_parameters_targets = dict()
    
    for key, value in target_parameters.iteritems():
        target_qi = (np.digitize(value, param_qbins[key], right=False).item() - 1)
        quantized_parameters_targets[key + "_qi"] = target_qi

    return filter_dataframe(df, quantized_parameters_targets)

In [ ]:
# Extract data
num_repetitions = generator_module.num_repetitions
parameter_names_sorted = data_pbs.dataset_infos['parameters']
all_args_arr = np.array(data_pbs.loaded_data['args_list'])

T_space = data_pbs.loaded_data['parameters_uniques']['T']
M_space = data_pbs.loaded_data['parameters_uniques']['M']
ratio_conj_space = data_pbs.loaded_data['parameters_uniques']['ratio_conj']

pandas_columns_with_processing = [
    ('result_all_precisions', dict(name='precision', process=avg_lastaxis)),
    ('result_FI_rc_curv', dict(name='fi_curv', process=process_fi)),
    ('result_FI_rc_theo', dict(name='fi_theo', process=process_fi)),
    ('result_FI_rc_theocov', dict(name='fi_theo_cov', process=process_fi)),
    ('result_marginal_FI', dict(name='fi_marginal', process=process_marginal_fi)),
    ('result_marginal_inv_FI', dict(name='inv_fi_marginal', process=process_marginal_fi)),
    ('result_em_fits', dict(name='emfit', process=process_em_fits)),
]

df_all_fits = construct_pandas_dataframe(data_pbs, pandas_columns_with_processing, num_repetitions)
df_all_fits.loc[:, ('T')] = df_all_fits.loc[:, ('T')].astype(int)
df_all_fits.loc[:, ('M')] = df_all_fits.loc[:, ('M')].astype(int)
df_all_fits.loc[:, ('fi_kappa_ratio')] = df_all_fits['fi_theo']/(2*df_all_fits['emfit_kappa'])
df_all_fits.loc[:, ('fi_stddev_ratio')] = df_all_fits['fi_theo_stddev']/(df_all_fits['emfit_stddev'])
df_all_fits.loc[:, ('margfi_stddev_ratio')] = df_all_fits['inv_fi_marginal_stddev']/(df_all_fits['emfit_stddev'])

df_all_fits = df_all_fits[df_all_fits['inv_fi_marginal_stddev'] < 1.5*np.pi]

In [ ]:
df_all_fits.describe()

In [ ]:
# Remove outliers
df_fits_filtered = remove_outliers(df_all_fits, 10)

In [ ]:
df_fits_filtered.describe()

## -> Some stats

In [ ]:
## Correlations
df_fits_filtered.corr()

In [ ]:
# Cross-correlation plots
g = sns.pairplot(df_fits_filtered,
             vars=['fi_theo_stddev', 'fi_marginal_stddev', 'inv_fi_marginal_stddev', 
                   'fi_curv_stddev', 'emfit_stddev'],
            )
for ax in g.axes.flat:  
    plt.setp(ax.get_xticklabels(), rotation=45)


In [ ]:
# Check parameters effects
sns.pairplot(df_fits_filtered,
             x_vars=parameter_names_sorted,
             y_vars=['fi_theo', 'fi_marginal', 'fi_curv', 'emfit_kappa', 'precision']
            )

In [ ]:
# for T, subdf in df_fits_filtered.groupby('T'):
#     g = sns.pairplot(subdf,
#                  x_vars=['fi_theo', 'fi_marginal', 'fi_curv', 'emfit_kappa'],
#                  y_vars=['fi_theo', 'fi_marginal', 'fi_curv', 'emfit_kappa'],
#                 )
#     g.fig.suptitle("T : %d "% T, fontsize=30)
#     for ax in g.axes.flat:  
#         plt.setp(ax.get_xticklabels(), rotation=45)


# -----   T=1 -----




## Quantize parameters

In [ ]:
nQuantiles = 20
parameters = ['M', 'ratio_conj']

df_singleitem = filter_dataframe(df_all_fits, dict(T=1))
df_quantized, param_qbins, param_qbins_middle = df_add_quantize_parameters(df_singleitem, parameters, nQuantiles)

### trying quantized Dataframes

In [ ]:
df_fixedM = filter_quantized_param(df_quantized, 
                                   dict(M=150), 
                                   param_qbins
                                  )

In [ ]:
df_fixedM[['fi_theo', 'emfit_stddev', 'emfit_target', 'M', 'ratio_conj']].corr()

In [ ]:
sns.pairplot(df_fixedM,
             x_vars=['M', 'ratio_conj'],
             y_vars=['emfit_stddev', 'emfit_target']
            )

## Create 2d plots M/ratio

In [ ]:
df_Mratio_effect = df_quantized.pivot_table(index='M_qi', columns='ratio_conj_qi', aggfunc='mean')

In [ ]:
cmap = sns.light_palette("green", as_cmap=True)

ax = sns.heatmap(df_Mratio_effect['emfit_stddev'].T, 
                 yticklabels=["%.2f" % v for v in param_qbins_middle['ratio_conj']],
                 xticklabels=["%d" % v for v in param_qbins_middle['M']],
                 cmap=cmap
                )
ax.invert_yaxis()
ax.set_ylabel("ratio conj")
ax.set_xlabel("M")

In [ ]:
target_stddev = utils.kappa_to_stddev(100)
kappa_evolution = df_Mratio_effect['emfit_stddev']/target_stddev
kappa_evolution[kappa_evolution < 0.1] = 0.1
kappa_evolution[kappa_evolution > 5.] = 5.

cmap = sns.diverging_palette(145, 280, s=85, l=25, n=10, as_cmap=True)
ax, im = utils.pcolor_2d_data(kappa_evolution, 
                              x=param_qbins_middle['M'],
                              y=param_qbins_middle['ratio_conj'], 
                              xlabel='M', 
                              xlabel_format="%d",
                              ylabel='ratio conj', 
                              vmin=0.5,
                              vmax=1.5,
                              cmap=cmap
                             )
ax.grid('off')

In [ ]:
fi_kappa_ratio = df_Mratio_effect['fi_theo_stddev']/(df_Mratio_effect['emfit_stddev'])
fi_kappa_ratio[fi_kappa_ratio > 1.5] = 1.5
fi_kappa_ratio[fi_kappa_ratio < 0.5] = 0.5

cmap = sns.diverging_palette(145, 280, s=85, l=25, n=7, as_cmap=True)
ax, im = utils.pcolor_2d_data(fi_kappa_ratio, 
                              x=param_qbins_middle['M'],
                              y=param_qbins_middle['ratio_conj'], 
                              xlabel='M', 
                              xlabel_format="%d",
                              ylabel='ratio conj', 
                              vmin=0.5,
                              vmax=1.5,
                              cmap=cmap
                             )
ax.grid('off')

## Interpolate 2D plots instead

In [ ]:
def add_target_kappa_columns(df, target_kappa=100):
    target_kappa_ratio = df['emfit_kappa']/target_kappa
    target_kappa_ratio[target_kappa_ratio > 5.] = 5.
    target_kappa_ratio[target_kappa_ratio < 0.01] = 0.01

    target_fi_ratio = df['fi_theo']/(2*target_kappa)
    target_fi_ratio[target_fi_ratio > 5.] = 5.
    target_fi_ratio[target_fi_ratio < 0.01] = 0.01

    df.loc[:, 'target_kappa_ratio'] = target_kappa_ratio
    df.loc[:, 'target_fi_ratio'] = target_fi_ratio

def add_target_stddev_columns(df, target_stddev=0.2):
    target_stddev_ratio = df['emfit_stddev']/target_stddev
    target_stddev_ratio[target_stddev_ratio > 5.] = 5.
    target_stddev_ratio[target_stddev_ratio < 0.01] = 0.01

    target_fi_stddev_ratio = df['fi_theo_stddev']/(target_stddev)
    target_fi_stddev_ratio[target_fi_stddev_ratio > 5.] = 5.
    target_fi_stddev_ratio[target_fi_stddev_ratio < 0.01] = 0.01
    
    target_margfi_stddev_ratio = df['inv_fi_marginal_stddev']/(target_stddev)
    target_margfi_stddev_ratio[target_margfi_stddev_ratio > 5.] = 5.
    target_margfi_stddev_ratio[target_margfi_stddev_ratio < 0.01] = 0.01
    
    df.loc[:, 'target_stddev_ratio'] = target_stddev_ratio
    df.loc[:, 'target_fi_stddev_ratio'] = target_fi_stddev_ratio
    df.loc[:, 'target_margfi_stddev_ratio'] = target_margfi_stddev_ratio

In [ ]:
target_kappa = 30
add_target_kappa_columns(df_quantized, target_kappa=target_kappa)
add_target_stddev_columns(df_quantized, target_stddev=utils.kappa_to_stddev(target_kappa))

In [ ]:
M_int_space = np.sort(df_quantized['M'].unique())
ratio_int_space = np.sort(df_quantized['ratio_conj'].unique())

In [ ]:
def compute_spline_interpolation(df, interpolate_column, x_col='', y_col='', kx=3, ky=3, s=None):
    return spint.SmoothBivariateSpline(df[x_col], 
                          df[y_col],
                          df[interpolate_column],
                          kx=kx, ky=ky, s=s)

In [ ]:
def plot_interpolated_Mratio(df, target_column, 
                             x_col='M', y_col='ratio_conj', 
                             title='', 
                             vmin=1.5, vmax=0.5, 
                             cmap='RdBu_r'):
    x_int_space = M_int_space = np.sort(df[x_col].unique())
    y_int_space = np.sort(df[y_col].unique())

    spline_int = compute_spline_interpolation(df, target_column, 
                                              x_col=x_col, y_col=y_col
                                             )
    return utils.pcolor_2d_data(spline_int(x_int_space, y_int_space),
                                x=x_int_space, 
                                y=y_int_space, 
                                xlabel=x_col, 
                                xlabel_format="%d", 
                                ylabel=y_col, 
                                title=title,
                                ticks_interpolate=11,
                                vmin=vmin,
                                vmax=vmax,
                                log_scale=False, 
                                cmap=cmap
                               )

In [ ]:
cmap_div = sns.diverging_palette(h_neg=29, h_pos=265, s=80, l=85, sep=10, as_cmap=True, center='dark')
plot_interpolated_Mratio(df_quantized, 'target_kappa_ratio', title='Ratio kappa/%.2f' % (target_kappa), cmap=cmap_div)
plot_interpolated_Mratio(df_quantized, 'target_fi_ratio', title='Ratio FI/%.2f' % (2*target_kappa), cmap=cmap_div)
plot_interpolated_Mratio(df_quantized, 'fi_kappa_ratio', title='Ratio FI/(2 Kappa)')

In [ ]:
cmap_div = sns.diverging_palette(h_neg=29, h_pos=265, s=80, l=85, sep=10, as_cmap=True, center='dark')
plot_interpolated_Mratio(df_quantized, 'target_stddev_ratio', 
                         title='Ratio stddev/%.2f' % (target_stddev), 
                        )
plot_interpolated_Mratio(df_quantized, 'target_fi_stddev_ratio', 
                         title='Ratio FI/%.2f' % (target_stddev), 
                        )
plot_interpolated_Mratio(df_quantized, 'target_margfi_stddev_ratio', 
                         title='Ratio Marginal Inv FI/%.2f' % (target_stddev), 
                        )
plot_interpolated_Mratio(df_quantized, 'fi_stddev_ratio', title='Ratio FI / Stddev')
plot_interpolated_Mratio(df_quantized, 'margfi_stddev_ratio', title='Ratio Marginal FI / Stddev')

In [ ]:
sns.pairplot(df_quantized,
             x_vars=['M', 'ratio_conj'],
             y_vars=['fi_stddev_ratio', 'margfi_stddev_ratio', 'emfit_stddev', 'emfit_LL']
            )

# --------- T = 2  -----------

In [ ]:
df_quantized_2, param_qbins, param_qbins_middle = df_add_quantize_parameters(filter_dataframe(df_all_fits, dict(T=2)), 
                                                                           parameters, 
                                                                           nQuantiles)

In [ ]:
target_kappa = 10
add_target_kappa_columns(df_quantized_2, target_kappa=target_kappa)
add_target_stddev_columns(df_quantized_2, target_stddev=utils.kappa_to_stddev(target_kappa))

In [ ]:
cmap_div = sns.diverging_palette(h_neg=29, h_pos=265, s=80, l=85, sep=10, as_cmap=True, center='dark')
plot_interpolated_Mratio(df_quantized_2, 'target_kappa_ratio', title='Ratio kappa/%.2f' % (target_kappa), cmap=cmap_div)
plot_interpolated_Mratio(df_quantized_2, 'target_fi_ratio', title='Ratio FI/%.2f' % (2*target_kappa), cmap=cmap_div)
plot_interpolated_Mratio(df_quantized_2, 'fi_kappa_ratio', title='Ratio FI/(2 Kappa)')

In [ ]:
cmap_div = sns.diverging_palette(h_neg=29, h_pos=265, s=80, l=85, sep=10, as_cmap=True, center='dark')
plot_interpolated_Mratio(df_quantized_2, 'target_stddev_ratio', 
                         title='Ratio stddev/%.2f' % (target_stddev), 
                        )
plot_interpolated_Mratio(df_quantized_2, 'target_fi_stddev_ratio', 
                         title='Ratio FI/%.2f' % (target_stddev), 
                        )
plot_interpolated_Mratio(df_quantized_2, 'target_margfi_stddev_ratio', 
                         title='Ratio Marginal Inv FI/%.2f' % (target_stddev), 
                        )
plot_interpolated_Mratio(df_quantized_2, 'fi_stddev_ratio', title='Ratio FI / Stddev')
plot_interpolated_Mratio(df_quantized_2, 'margfi_stddev_ratio', title='Ratio Marginal FI / Stddev')

In [ ]:
sns.pairplot(df_quantized_2,
             x_vars=['M', 'ratio_conj'],
             y_vars=['fi_stddev_ratio', 'emfit_stddev', 'emfit_target', 'emfit_LL']
            )

In [ ]:
## Ratio Conj 0.7 is weird ?

df_ratioproblem = filter_quantized_param(df_quantized_2, dict(ratio_conj=0.7), param_qbins)

g = sns.lmplot(data=df_ratioproblem, x='M', y='emfit_stddev', truncate=True, order=2, fit_reg=False)
xlims = (df_ratioproblem['M'].min(), df_ratioproblem['M'].max())
g.set(xlim=xlims, ylim=(0, 0.8))

g = sns.lmplot(data=df_ratioproblem, x='M', y='fi_theo_stddev', truncate=True, order=2, fit_reg=False)
g.set(xlim=xlims, ylim=(0, 0.8))

g = sns.lmplot(data=df_ratioproblem, x='M', y='inv_fi_marginal_stddev', truncate=True, order=2, fit_reg=False)
g.set(xlim=xlims, ylim=(0, 0.8))


g = sns.lmplot(data=df_ratioproblem, x='M', y='fi_stddev_ratio', truncate=True, order=2, fit_reg=False,
               scatter_kws={'color': 'red'}
              )
g.set(xlim=xlims, ylim=(0.5, 1.5))

g = sns.lmplot(data=df_ratioproblem, x='M', y='margfi_stddev_ratio', truncate=True, order=2, fit_reg=False,
               scatter_kws={'color': 'red'}
              )
g.set(xlim=xlims, ylim=(0.5, 1.5))

In [ ]:
## Check Small M effect

df_smallM = filter_quantized_param(df_quantized_2, dict(M=10), param_qbins)

g = sns.lmplot(data=df_smallM, x='ratio_conj', y='emfit_stddev', order=2, fit_reg=False)
xlims = (df_smallM['ratio_conj'].min(), df_smallM['ratio_conj'].max())
g.set(xlim=xlims, ylim=(0, 1.5))

g = sns.lmplot(data=df_smallM, x='ratio_conj', y='fi_theo_stddev', order=2, fit_reg=False)
g.set(xlim=xlims, ylim=(0, 1.5))

g = sns.lmplot(data=df_smallM, x='ratio_conj', y='inv_fi_marginal_stddev', order=2, fit_reg=False)
g.set(xlim=xlims, ylim=(0, 1.5))

g = sns.lmplot(data=df_smallM, x='ratio_conj', y='fi_stddev_ratio', order=2, fit_reg=False,
               scatter_kws={'color': 'red'}
              )
g.set(xlim=xlims, ylim=(0.5, 1.5))

g = sns.lmplot(data=df_smallM, x='ratio_conj', y='margfi_stddev_ratio', truncate=True, order=2, fit_reg=False,
               scatter_kws={'color': 'red'}
              )
g.set(xlim=xlims, ylim=(0.5, 1.5))